In [1]:
from __future__ import print_function

import SimpleITK as sitk
print(sitk.Version())

# import convenient method for inline displaying from sitk to pylab 
from myshow import myshow3d
from myshow import myshow

from ipywidgets import widgets
from ipywidgets import interact, interactive
from math import pi

import numpy as np
import os
import sys

import matplotlib.pyplot as plt
%matplotlib inline

# Download data to work on
from downloaddata import fetch_midas_data as fdata

import tempfile

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [2]:
def prepare_resampled_values(fixed,moving,transform):
    """ Resample moving into space of fixed, and then rescale the
    intensity values of both to fit in a UInt8 data type.
    """
    f = sitk.ResampleImageFilter()
    f.SetTransform(transform)
    f.SetReferenceImage(fixed)
    print(f.GetTransform)
    deformed_moving = f.Execute(moving)
    fixed = sitk.Cast(sitk.RescaleIntensity(fixed),sitk.sitkUInt8)
    deformed_moving = sitk.Cast(sitk.RescaleIntensity(deformed_moving),sitk.sitkUInt8)
    return fixed, deformed_moving

def cshow(fixed, moving, transform=sitk.Transform(), title=""):
    """Show a single center slice inline in the notebooks of the rescaled images.
    """
    rescaled_fixed, deformed_rescaled_moving = prepare_resampled_values( fixed, moving, transform)
    myshow(sitk.Compose(deformed_rescaled_moving, deformed_rescaled_moving, sitk.Maximum(deformed_rescaled_moving,rescaled_fixed)), title=title)

def cshow3d(fixed, moving, transform=sitk.Transform(), title=""):
    """Show a tile of slices inline in the notebooks.
    """
    rescaled_fixed, deformed_rescaled_moving = prepare_resampled_values( fixed, moving, transform)
    myshow3d(sitk.Compose(deformed_rescaled_moving, deformed_rescaled_moving, sitk.Maximum(deformed_rescaled_moving,rescaled_fixed)), zslices=range(0, deformed_rescaled_moving.GetSize()[2],5), dpi=8,title=title)

In [3]:
def command_iteration(method) :
    print("{0} = {1} : {2}".format(method.GetOptimizerIteration(),
                                   method.GetMetricValue(),
                                   method.GetOptimizerPosition()))
    sys.stdout.flush()

def command_iteration_bsplineAndDisplacment(method) :
    print("{0} = {1}".format(method.GetOptimizerIteration(),
                                   method.GetMetricValue()))
    sys.stdout.flush()
    
def command_save_start(method):
        global cmd_values, cmd_positions
        cmd_values = np.array([])
        cmd_positions = np.ndarray(shape=(0,len(method.GetInitialTransform().GetParameters())))
    
def command_save_iteration(method):
    global cmd_values, cmd_positions
    cmd_values = np.concatenate((cmd_values,[method.GetMetricValue()]))
    cmd_positions = np.concatenate((cmd_positions,[method.GetOptimizerPosition()]),axis=0)
    

def plot2DTranslation(values, positions, figtitle=""):
    from matplotlib.font_manager import FontProperties
    
    fig, (ax0, ax1) = plt.subplots(nrows=2)
    
    
    if figtitle != "":
        fig.text(0.5, 0.95, figtitle,
               horizontalalignment='center',
               fontproperties=FontProperties(size=16))
    ax0.plot(range(len(values)),values)
    ax0.set_title('Metric Value')
    ax0.set_xlabel('value')
    ax0.set_ylabel('iteration #')

    ax1.plot(positions[:,-1],positions[:,-2])
    ax1.set_title('Position')
    ax1.set_xlabel('x translation')
    ax1.set_ylabel('y translation')

    plt.tight_layout()
    plt.show()

In [4]:
def return_my_name(imgFile,prefix="None",suffix=".png") :
    fileName = tempfile.NamedTemporaryFile(prefix=prefix,suffix=suffix,delete=False)
    sitk.WriteImage(imgFile,fileName.name)
    return fileName.name
    

def run_test_driver(baseline, test, numberOfPixelsTolerance, radiusTolerance, intensityTolerance):
    TEST_DRIVER_ROOT = "/Users/aghayoor/WorkSpace/SimpleITK_DIRS/SITK/release/ITK-prefix/bin"
    TEST_DRIVER = TEST_DRIVER_ROOT+"/itkTestDriver"
    file1 = return_my_name(baseline,"base",".nii.gz")
    file2 = return_my_name(test,"test",".nii.gz")
    !$TEST_DRIVER --no-process \
    --compareNumberOfPixelsTolerance $numberOfPixelsTolerance \
    --compareRadiusTolerance $radiusTolerance \
    --compareIntensityTolerance $intensityTolerance \
    --compare $file1 $file2
    print("Exit code: ".format(_exit_code))
    return _exit_code
 

def compare_with_baseline(fixed, moving, baseline, transform=sitk.Transform(),
                          numberOfPixelsTolerance=0,
                          radiusTolerance=0,
                          intensityTolerance=0):
    f = sitk.ResampleImageFilter()
    f.SetTransform(transform)
    f.SetReferenceImage(fixed)
    out = f.Execute(moving)
    ret = run_test_driver(baseline, out, numberOfPixelsTolerance, radiusTolerance, intensityTolerance)
    if ret:
        print("Test failed. ")
        #diff = baseline-out
        #myshow3d(diff, zslices=[out.GetSize()[2]/2], dpi=10,title=title)
    else:
        print("Test passed. ")
